In [1]:
# -*- coding: utf-8
# The Intercept Brasil (https://theintercept.com/brasil/)
# Reinaldo Chaves (@paidatocandeira)
# Soma as empresas de segurança da Receita com as indicadas pela Polícia Federal
#

In [2]:
import pandas as pd
import unidecode

Função para retirar acentuação 

In [3]:
def f(str):
    return (unidecode.unidecode(str))

O arquivo gerado em '1_roboseguranca_socios.ipynb' foi checado, limpo e preenchido manualmente. Carrega o resultado

Foi decido trabalhar apenas com empresas de RIO DE JANEIRO, NOVA IGUAÇU, SÃO GONÇALO, NITERÓI, DUQUE DE CAXIAS e SÃO JOÃO DE MERITI

In [6]:
consolidado = pd.read_excel('empresas_de_seguranca_consolidado_etapa1.xlsx', converters={'cnpj': lambda x: str(x), 'numero_logradouro': lambda x: str(x), 'complemento': lambda x: str(x)} )

In [7]:
consolidado.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1224 entries, 0 to 1223
Data columns (total 25 columns):
cnpj                                1224 non-null object
CNAE_principal                      1134 non-null object
nome_empresa                        1224 non-null object
nome_empresa_maiuscula_minuscula    999 non-null object
estado                              999 non-null object
indicador                           1145 non-null float64
documento                           1145 non-null object
qualificacao                        1145 non-null object
socio                               1148 non-null object
socio_maiuscula_minuscula           1002 non-null object
data_abertura                       1216 non-null object
logradouro                          1153 non-null object
numero_logradouro                   1153 non-null object
complemento                         644 non-null object
cep                                 938 non-null object
bairro                              1204 

Função para preencher zeros à esquerda

In [8]:
def compara2(linha):
    if len(linha['cnpj']) < 14:
        conta = 14 - (len(linha['cnpj']))
        zero = '0'
        zeros = zero * conta
        alterado = zeros + linha['cnpj']
    else:
        alterado = linha['cnpj']
    return alterado

In [9]:
nova = consolidado.apply(compara2, axis=1) #lê dataframe e chama a função

consolidado['cnpj_inteiro'] = nova 

Filtra apenas empresas de seguranca privada que tenham sócios ou administradores vindo de forças de segurança

In [10]:
policiais = consolidado[(consolidado['cargo_relacionado_socio'] == 'Policia Civil') | 
                        (consolidado['cargo_relacionado_socio'] == 'Policia Militar') | 
                        (consolidado['cargo_relacionado_socio'] == 'Marinha') | 
                        (consolidado['cargo_relacionado_socio'] == 'Bombeiros') | 
                        (consolidado['cargo_relacionado_socio'] == 'Secretario de Seguranca') | 
                        (consolidado['cargo_relacionado_socio'] == 'Exercito') | 
                        (consolidado['cargo_relacionado_socio'] == 'Administracao Penitenciaria') | 
                        (consolidado['cargo_relacionado_socio'] == 'Aeronautica')]

In [11]:
policiais.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 140 entries, 8 to 1222
Data columns (total 26 columns):
cnpj                                140 non-null object
CNAE_principal                      124 non-null object
nome_empresa                        140 non-null object
nome_empresa_maiuscula_minuscula    117 non-null object
estado                              117 non-null object
indicador                           133 non-null float64
documento                           133 non-null object
qualificacao                        133 non-null object
socio                               135 non-null object
socio_maiuscula_minuscula           120 non-null object
data_abertura                       139 non-null object
logradouro                          131 non-null object
numero_logradouro                   131 non-null object
complemento                         75 non-null object
cep                                 94 non-null object
bairro                              140 non-null object

In [40]:
policiais.to_csv('empresas_policiais_e_cia.csv', index=False)

# PF

No Brasil as empresas de segurança privada precisam de registro na Polícia Federal para funcionarem. Conseguimos essa lista de empresas com a PF

In [13]:
lista_pf = pd.read_csv('empresas_seg_br_mar18_pf.csv', sep=',',encoding = 'utf-8', converters={'CNPJ': lambda x: str(x)} )

In [14]:
def compara(linha):
    if len(linha['CNPJ']) < 14:
        conta = 14 - (len(linha['CNPJ']))
        zero = '0'
        zeros = zero * conta
        alterado = zeros + linha['CNPJ'] 
    else:
        alterado = linha['CNPJ'] 
    return alterado

nova = lista_pf.apply(compara, axis=1) #lê dataframe e chama a função
lista_pf['cnpj_inteiro'] = nova # adiciona coluna com os resultados ao dataframe

Seleciona apenas as do RJ

In [15]:
lista_pf_rj = lista_pf[(lista_pf['Agrupamento'] == 'RJ')]

In [16]:
conta_cnpj = len(lista_pf_rj['CNPJ'].value_counts())
conta_cnpj

336

Filtra por cidades

In [17]:
lista_pf_rj = lista_pf_rj[(lista_pf_rj['Município'] == 'RIO DE JANEIRO') |
                          (lista_pf_rj['Município'] == 'NOVA IGUAÇU') | 
                          (lista_pf_rj['Município'] == 'SÃO GONÇALO') | 
                          (lista_pf_rj['Município'] == 'NITERÓI') | 
                          (lista_pf_rj['Município'] == 'DUQUE DE CAXIAS') | 
                          (lista_pf_rj['Município'] == 'SÃO JOÃO DE MERITI')]

In [18]:
lista_pf_rj.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 259 entries, 2920 to 3269
Data columns (total 13 columns):
Agrupamento            259 non-null object
Grupo                  259 non-null object
Tipo                   259 non-null object
CNPJ                   259 non-null object
Razão Social           259 non-null object
E-Mail                 256 non-null object
Município              259 non-null object
UF                     259 non-null object
Nível                  259 non-null object
Tipo de Empresa        183 non-null object
Tipos de Atividades    259 non-null object
Situação               259 non-null object
cnpj_inteiro           259 non-null object
dtypes: object(13)
memory usage: 28.3+ KB


Cruza a lista policiais, que veio da Receita, com a a lista da PF. Isso vai permitir acrescentar as empresas que possam ter ficado de fora na pesquisa na base da Receita

In [22]:
encontrados = pd.merge(policiais, lista_pf_rj, left_on='cnpj_inteiro', right_on='cnpj_inteiro')

In [23]:
encontrados.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 31 entries, 0 to 30
Data columns (total 38 columns):
cnpj                                31 non-null object
CNAE_principal                      31 non-null object
nome_empresa                        31 non-null object
nome_empresa_maiuscula_minuscula    30 non-null object
estado                              30 non-null object
indicador                           30 non-null float64
documento                           30 non-null object
qualificacao                        30 non-null object
socio                               30 non-null object
socio_maiuscula_minuscula           31 non-null object
data_abertura                       31 non-null object
logradouro                          29 non-null object
numero_logradouro                   29 non-null object
complemento                         6 non-null object
cep                                 20 non-null object
bairro                              31 non-null object
municipio        

In [21]:
encontrados.to_csv('encontrados_lista_pf_consolidado.csv', index=False)

In [28]:
nomes_naoencontrados = pd.merge(policiais, 
                        lista_pf_rj, 
                        left_on='cnpj_inteiro', 
                        right_on='cnpj_inteiro',
                        how='outer',
                        indicator=True)

rdf = nomes_naoencontrados.query("_merge == 'right_only'").drop('_merge',axis=1)

rdf.info()
rdf.CNPJ.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 236 entries, 140 to 375
Data columns (total 38 columns):
cnpj                                0 non-null object
CNAE_principal                      0 non-null object
nome_empresa                        0 non-null object
nome_empresa_maiuscula_minuscula    0 non-null object
estado                              0 non-null object
indicador                           0 non-null float64
documento                           0 non-null object
qualificacao                        0 non-null object
socio                               0 non-null object
socio_maiuscula_minuscula           0 non-null object
data_abertura                       0 non-null object
logradouro                          0 non-null object
numero_logradouro                   0 non-null object
complemento                         0 non-null object
cep                                 0 non-null object
bairro                              0 non-null object
municipio                   

140     5014372000190
141    15454089000133
142     2841990000116
143    16599942000178
144     7028291000256
Name: CNPJ, dtype: object

In [27]:
rdf.to_csv('naoencontrados_lista_pf_consolidado.csv', index=False)

Esses CNPJs não encontrados serão checados manualmente da mesma forma no site da Receita e serão depois inseridos na lista consolidado